In [1]:
!pip install sentence_transformers

     |████████████████████████████████| 79 kB 3.2 MB/s 
     |████████████████████████████████| 4.0 MB 37.3 MB/s 
     |████████████████████████████████| 1.2 MB 27.8 MB/s 
     |████████████████████████████████| 77 kB 2.3 MB/s 
     |████████████████████████████████| 6.6 MB 31.2 MB/s 
     |████████████████████████████████| 596 kB 14.3 MB/s 
     |████████████████████████████████| 895 kB 51.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=d5cab9a457d9b5370cfe56ec3c08436252adab1565d06c75f42579f3e7aa820e
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [61]:
doc = """
      AZIR
Health growth increased.

 A little birdy told us that Azir's been weak, and the numbers told us the birdy was right. His lack of Pro presence in a scaling meta that should favor him was the final confirmation, so we're giving him some base stat power that should help the Emperor of Shurima in all skill brackets. 
BASE STATS
HEALTH GROWTH 92 ⇒ 105
HEALTH AT LEVEL 18 2116 ⇒ 2337

DARIUS
R base damage increased.

 Darius has been lagging behind other juggernauts after the fighter item changes, so we're giving him a beefy damage boost. (As a reminder, his ult's total damage can increase by up to 100% based on his passive stacks.) 
R - NOXIAN GUILLOTINE
MINIMUM TRUE DAMAGE 100/200/300 (+75% bonus AD) ⇒ 125/250/375 (+75% bonus AD)

HECARIM
Q and E base damage decreased late.

 Tankarim is taking over Pro again, so we're nerfing the base damage on some of his abilities to ensure fairer horseplay. 
Q - RAMPAGE
BASE DAMAGE 60/97/134/171/208 (+85% bonus AD) ⇒ 60/90/120/150/180 (+85% bonus AD)
E - DEVASTATING CHARGE
MINIMUM TOTAL DAMAGE 30/50/70/90/110 (+55% bonus AD) ⇒ 30/45/60/75/90 (+55% bonus AD)
MAXIMUM TOTAL DAMAGE 60/100/140/180/220 (+110% bonus AD) ⇒ 60/90/120/150/180 (+110% bonus AD)

ILLAOI
QoL change on R and several bugfixes.

 No balance changes here; instead, we fixed lots of bugs and updated Illaoi's ult to give her some more control over her tentacles. 
R - LEAP OF FAITH
UPDATEDFACE ME R - Leap of Faith now casts with Illaoi facing toward your cursor, allowing for more favorable Tentacle spawn locations
BUGFIXES
When Illaoi attacks a Vessel while empowered by W - Harsh Lesson, nearby inactive Tentacles will now slam her target as soon as they become active (short delay removed)
Spell queueing has been added to W - Harsh Lesson and R - Leap of Faith, so you can now buffer other spells while casting them
W - Harsh Lesson's visual indicators have been fixed and will show the correct range of her empowered basic attacks
Tentacles can no longer spawn on either team's base gates on Summoner's Rift
Vessels no longer contribute to Illaoi's creep score when they expire or are destroyed
Q - Tentacle Smash and E - Test of Spirit will now be visible to nearby enemy champions when cast from Fog of War

JAX
Base health increased. W base damage increased.

 Jax has been mediocre for a while, so we're giving him some early game power to help him get to his item power spikes. He may not have a real weapon at first, but now he can buy one a little sooner. :^) 
BASE STATS
BASE HEALTH 593 ⇒ 615
W - EMPOWER
MAGIC DAMAGE 40/75/110/145/180 (+60% AP) ⇒ 50/85/120/155/190 (+60% AP)

NIDALEE
Human W cost decreased; E mana cost decreased late and cast range increased. Cougar W damage radius increased.

 Human-form Nidalee is a mana-hungry huntress, but her overall kit is much more compatible with the non-mana mage Mythics. We're giving her more access to her utility spells as well as making her AoE range bigger so she can claw enemies and jungle camps more effectively. Lastly, her heal cast range was pretty restrictive (especially for a character with so many other positioning constraints), so we're increasing it to match that of her Bushwhack trap placement. All in all, this should make her more accessible and stronger across all skill brackets. 
HUMAN FORM
W - BUSHWHACK
MANA COST 40/45/50/55/60 ⇒ 30/35/40/45/50
E - PRIMAL SURGE
MANA COST 50/60/70/89/90 ⇒ 50/55/60/65/70
CAST RANGE 600 ⇒ 900 (now matches W - Bushwhack)
COUGAR FORM
W - POUNCE
AOE DAMAGE RADIUS 225 ⇒ 250

RENGAR
Passive adjustments to Ferocity, leaps, and Bonetooth Necklace; new Ferocity resource bar added. Q basic attacks now always crit; damage now scales with crit chance; now applies against towers; no longer applies against plants. E cast time during leap removed; now grants true sight and some vision around target. R now also grants some vision around target.


 Our big kitty finally has some big changes coming to his kit! Our goals were to smooth out Rengar's gameplay, keep his current playstyle, and clarify his builds. We're going to keep a close eye on the Q crit damage ratio so he doesn't get too out of line. We also know that Q no longer applying on plants can be painful for seasoned players who used this mechanic to optimize Ferocity stacks, but we strongly believe it wasn't healthy to support long term and have instead moved that power to more accessible parts of his kit. 
PASSIVE - UNSEEN PREDATOR
UPDATEDHERE, KITTY KITTY Rengar's next leap will grant 1 Ferocity only if he has 0 Ferocity ⇒ upon losing all Ferocity stacks (no matter how much Ferocity he gains before his next leap)
UPDATEDPURRFECT POUNCES Rengar will be able to leap after 0.35 seconds in brush or camouflage (or Senna's E - Curse of the Black Mist camouflage) consistently, rather than based on a 0.3-0.45 second timer (Leap range has been slightly increased to offset variability)
NEWFEROCIOUS FELINE Rengar now has a Ferocity resource bar that indicates how much Ferocity he currently has (Stacks generated by leaps will have a different color so you'll know if you'll gain Ferocity on your next leap)
ALL FEROCITY STACKS FALL OFF AFTER 8 seconds ⇒ 10 seconds out of combat
BONETOOTH NECKLACE TAKEDOWN TIMER Within 1.5 seconds ⇒ 3 seconds of damaging an enemy champion
Q - SAVAGERY
NEWSAVAGE STRIKES Rengar's Q-empowered basic attack is now always a critical strike. Instead of regular critical strike damage, each 1% critical strike chance increases this empowered attack's damage by 0.66% (0.99% with Infinity Edge's Perfection passive)
UPDATEDSCRATCHING POSTS Q - Savagery now also applies to basic attacks against towers
UPDATEDTHAT WASN'T CATNIP Q - Savagery no longer applies to basic attacks against plants
E - BOLA STRIKE
UPDATEDNO CAST, ONLY THROW Rengar can now throw his E - Bola Strike instantly (with zero cast time) during leaps
NEWPEEKABOO Now grants true sight and normal vision 150 units around the first enemy hit for 2 seconds
R - THRILL OF THE HUNT
UPDATEDPREDATORY INSTINCT Now grants not only true sight, but also normal vision 100 units around the nearest enemy

TRYNDAMERE
E cooldown reduction per crit decreased. R cooldown increased.

 Tryndamere is too strong at the highest levels of play. His kit scales too well with Ability Haste, so we're readjusting his cooldowns to account for the higher amounts of Ability Haste available to him through items. 
E - SPINNING SLASH
COOLDOWN REDUCTION PER CRITICAL STRIKE 1 second (2 seconds against champions) ⇒ 0.75 seconds (1.5 seconds against champions)
R - UNDYING RAGE
COOLDOWN 110/100/90 seconds ⇒ 130/110/90 seconds

GANGPLANK
Passive burn base damage increased late. W mana cost decreased.

 Gangplank is arrrguably struggling at all levels of play. He's already got enough gunpowder in his Q and barrels, so we're putting some power into less explosive and more… orange… parts of his kit. 
PASSIVE - TRIAL BY FIRE
BURN BASE DAMAGE 55-225 (levels 1-18) ⇒ 55-310 (levels 1-18)
W - REMOVE SCURVY
MANA COST 80/90/100/110/120 ⇒ 60/70/80/90/100

GWEN
E cooldown refund increased. R cooldown decreased early.

 Gwen's been hanging by a thread after her most recent nerfs, so we're stitching her up with some cooldown buffs that should help her across all skill levels. Needle-ss to say, we don't think these changes will fully address her issues, but we're being extra careful considering her pattern of dominating Pro. 
E - SKIP ‘N SLASH
COOLDOWN REFUND UPON FIRST BASIC ATTACK 50% ⇒ 60%
R - NEEDLEWORK
COOLDOWN 120/100/80 seconds ⇒ 100/90/80 seconds

JAYCE
Base health decreased, health growth increased, and armor adjusted.

 Jayce has also been a popular pick in Pro. We're lowering his laning power by making him squishier early on, but increasing his growth to make sure he still scales to late game. 
BASE STATS
HEALTH GROWTH 90 ⇒ 95
BASE HEALTH 560 ⇒ 520
HEALTH 560-2090 (levels 1-18) ⇒ 520-2135 (levels 1-18)
BASE ARMOR 27 ⇒ 22
ARMOR GROWTH 3.5 ⇒ 3.8 (armor at level 18 unchanged)

KALISTA
Base mana, move speed, and attack speed growth increased.

 Kalista hasn’t been seen often in Pro this season, and is generally weak in all levels of play. We're buffing some of her baseline stats which should especially help her out in Average play. Super speedy spooky spears incoming. 
BASE STATS
BASE MANA 250 ⇒ 300
MOVE SPEED 325 ⇒ 330
ATTACK SPEED GROWTH 4% ⇒ 4.5%

KARTHUS
Q damage against monsters increased.

 While the Deathsinger remains powerful in bot lane, his voice has been faltering in the jungle. We're amping up his clear strength to bring jungle Karthus back from the dead. 
Q - LAY WASTE
DAMAGE AGAINST MONSTERS 85% ⇒ 95%

LEE SIN
Base AD decreased. Q base damage decreased.

 We're looking (no pun intended) to bring down Lee Sin's jungle clear speed and early game threat in order to reduce his impact on Pro play. We're targeting his base damage instead of his cooldowns to ensure his passive's rhythm remains the same. 
BASE STATS
BASE AD 68 ⇒ 66
Q - SONIC WAVE / RESONATING STRIKE
SONIC WAVE DAMAGE 55/80/105/130/155 (+100% bonus AD) ⇒ 50/75/100/125/150 (+100% bonus AD)
RESONATING STRIKE MINIMUM DAMAGE 55/80/105/130/155 (+100% bonus AD) ⇒ 50/75/100/125/150 (+100% bonus AD)
RESONATING STRIKE MAXIMUM DAMAGE 110/160/210/260/310 (+200% bonus AD) ⇒ 100/150/200/250/300 (+200% bonus AD)

LILLIA
Passive healing against champions increased. W base damage increased.

 We're giving Lillia some more scaling sustain with her AP builds while also boosting her early jungle clear. These buffs should give her W damage a bit more oomph—or rather, eep! 
PASSIVE - DREAM-LADEN BOUGH
HEALING AGAINST CHAMPIONS 12-140 (levels 1-18) (+12% AP) ⇒ 12-140 (levels 1-18) (+20% AP)
W - WATCH OUT! EEP!
DAMAGE 70/90/110/130/150 (+35% AP) ⇒ 80/100/120/140/160 (+35% AP)

NEEKO
W passive base damage and move speed increased.

 Neeko's been hidden from Pro play despite being a strong tomato in solo queue. Her on-hit build was quite popular in past metas, so we're bringing it back to lure the Curious Chameleon out of hiding! 
W - SHAPESPLITTER
PASSIVE BONUS MAGIC DAMAGE 50/70/90/110/130 (+60% AP) ⇒ 50/80/110/140/170 (+60% AP)
PASSIVE BONUS MOVE SPEED 10/15/20/25/30% ⇒ 10/17.5/25/32.5/40%

PANTHEON
Base health regen decreased. Tap Q range increased and width decreased. E self-slow no longer applies when moving backwards. R crash now lands faster.

 Pantheon hasn't been living up to his Spartan spirit, so we're sharpening his spear to make him feel more like the paragon of Rakkor warriors. Skilled Pantheon players should now be able to showcase their strength with more timely ults, more maneuverable shields, and better trading tools. 
BASE STATS
BASE HEALTH REGENERATION 9 ⇒ 7.5
Q - COMET SPEAR
TAP Q LENGTH 550 units ⇒ 600 units
TAP Q WIDTH 150 units ⇒ 120 units
E - AEGIS ASSAULT
REMOVEDSTEADFAST SHIELDING Pantheon is no longer slowed if moving backwards while E - Aegis Assault is active
R - GRAND STARFALL
UPDATEDCRASH LANDING ON YOU Pantheon now crashes down on his destination 0.25 seconds sooner

RENGAR
Q bonus damage against structures reduced. W bonus damage against monsters increased.

 Rengar's become a serious destroyer of furniture structures after his most recent changes. This is a new and flexible strength in his kit that we want to preserve, but he's taking down those (scratching) towers a bit too fast, so we're tapping down the damage on that part of his Q. We're also bumping up his damage against monsters to compensate for some of the clear speed he lost because of the changes to Q against plants last patch. 
Q - SAVAGERY
NEWKITTEN MITTENS Q - Savagery's bonus damage is reduced by 40% against structures
W - BATTLE ROAR
BONUS MAGIC DAMAGE AGAINST MONSTERS 65-130 (levels 1-18) ⇒ 85-150 (levels 1-18)
BUGFIXES
Fixed a bug where Gromp did not grant Rengar a stack of Ferocity

RYZE
Q damage AP ratio increased; bonus mana ratio decreased.

 Ryze's tank builds make him a safe yet potent split pusher by allowing him to stack health while dealing carry levels of damage, making him particularly lethal in Pro play. The mid-patch 12.5 nerfs didn't quite do the job at refining his hypercarry identity, so we're shifting his mana scaling to incentivize him to build a few more AP items before stacking health items like Winter’s Approach (which will also be receiving nerfs this patch) . 
Q - OVERLOAD
MAGIC DAMAGE 70/90/110/130/150 (+45% AP) (+3% bonus mana) ⇒ 70/90/110/130/150 (+50% AP) (+2% bonus mana)

WUKONG
Q cooldown decreased. W cooldown decreased late; dash can now go through terrain. E AP ratio and damage against monsters increased.

 Wukong hasn't been seen in Pro much despite having a theoretically good matchup into meta staples like Jayce and Graves. We're helping him claim his place in top lane while also reinvigorating jungle Wu so he can be a viable counter or flex pick. 
Q - CRUSHING BLOW
COOLDOWN 9/8.5/8/7.5/7 seconds ⇒ 8/7.5/7/6.5/6 seconds
W - WARRIOR TRICKSTER
COOLDOWN 20/19/18/17/16 seconds ⇒ 20/18/16/14/12 seconds
NEWTO THE WINDOWS, THROUGH THE WALLS Wukong can now dash through terrain using W - Warrior Trickster
E - NIMBUS STRIKE
MAGIC DAMAGE 80/110/140/170/200 (+80% AP) ⇒ 80/110/140/170/200 (+100% AP)
BONUS MAGIC DAMAGE AGAINST MONSTERS 50% ⇒ 80%

YASUO
Base health increased. R cooldown decreased early.

 Despite my match history, Yasuo and Yone have been relatively weak (and the sustain nerfs in 12.6 didn’t do them any favors either). Rather than increasing their damage, we're giving the wind brothers more opportunity to flex their skills. For Yasuo, we're increasing his early combat durability and ability to synergize with teammates during laning phase. 
BASE STATS
HEALTH 490 ⇒ 520
R - LAST BREATH
COOLDOWN 80/55/30 seconds ⇒ 70/50/30 seconds

YONE
R cooldown decreased late.

 Yone is also getting more access to his ult, but with a focus on late game which is when it's usually harder for him to close gaps (unless he's already extremely far ahead). 
R - FATE SEALED
COOLDOWN 120/100/80 seconds ⇒ 120/90/60 seconds

ZERI
Base AD, AD growth, and armor decreased; health growth increased. Q base damage decreased, AD ratio increased; now applies on-hit effects only once per cast; now grants at most one R stack and E cooldown reduction once per cast, but crits can grant one more of each. W base damage decreased, AD and AP ratio increased. E cooldown increased early and decreases with rank again. R cooldown increased early and decreased late.

 Zeri, Zeri, you’ve been a little too unfair-y. Despite the nerfs in previous patches, Zeri has been shockingly resilient and continues to dominate Pro play. With this round of nerfs, we’re aiming to bring her bruiser build more in line with her crit build, as well as make her stacks and on-hits more predictable. This should give enemies more time to catch Zeri before she starts zipping circles around them. 
BASE STATS
AD GROWTH 2 ⇒ 1.5
ATTACK DAMAGE 53-87 (levels 1-18) ⇒ 50-75.5 (levels 1-18)
BASE ARMOR 22 ⇒ 20
HEALTH GROWTH 90 ⇒ 95
HEALTH AT LEVEL 18 2030 ⇒ 2115
Q - BURST FIRE
TOTAL PHYSICAL DAMAGE 10/15/20/25/30 (+110% total AD) ⇒ 7/9/11/13/15 (+110/112.5/115/117.5/120% total AD)
UPDATEDONCE UPON A CAST Q - Burst Fire now only applies on-hit effects once per cast
UPDATEDNOT SO FAST Zeri can now only gain one stack of R - Lightning Crash and one instance of E - Spark Surge cooldown reduction per Q - Burst Fire cast (excluding critical strikes)
NEWDOUBLE THE SHOCK Critical strikes on Q - Burst Fire grant Zeri an additional R - Lightning Crash stack and an additional instance of E - Spark Surge’s cooldown reduction (at most two procs per cast: one for Q upon hitting a champion and another if it crits on a champion)
W - ULTRASHOCK LASER
MAGIC DAMAGE 40/75/110/145/180 (+100% total AD)(+60% AP) ⇒ 10/45/80/115/150 (+120% total AD)(+70% AP)
E - SPARK SURGE
COOLDOWN 23 seconds ⇒ 28/26.5/25/23.5/22 seconds
R - LIGHTNING CRASH
COOLDOWN 100/90/80 seconds ⇒ 120/95/70 seconds
      """

In [62]:
n_gram_range = (1, 3)
stop_words = "english"

count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 3789
trigram 다섯개만 출력 : ['10' '10 15' '10 15 20' '10 17' '10 17 25']


In [63]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [64]:
top_n = 10
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['cast hitting champion', 'crits champion ultrashock', 'monsters increased wukong', 'bigger claw enemies', 'lagging juggernauts fighter', 'speedy spooky spears', 'monsters increased deathsinger', 'seconds combat bonetooth', 'champion ultrashock laser', 'increased jax mediocre']


In [41]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [65]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=10, nr_candidates=10)

['cast hitting champion',
 'crits champion ultrashock',
 'monsters increased wukong',
 'bigger claw enemies',
 'lagging juggernauts fighter',
 'speedy spooky spears',
 'monsters increased deathsinger',
 'seconds combat bonetooth',
 'champion ultrashock laser',
 'increased jax mediocre']

In [66]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=10, nr_candidates=20)

['jax mediocre giving',
 'changes giving beefy',
 'seconds damaging enemy',
 'fairer horseplay rampage',
 'recent nerfs stitching',
 'health regeneration comet',
 'crits champion ultrashock',
 'bigger claw enemies',
 'lagging juggernauts fighter',
 'champion ultrashock laser']

In [10]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [67]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=10, diversity=0.2)

['increased jax mediocre',
 'champion ultrashock laser',
 'seconds combat bonetooth',
 'lagging juggernauts fighter',
 'speedy spooky spears',
 'recent nerfs stitching',
 'health regeneration comet',
 'monsters increased deathsinger',
 'changes giving beefy',
 'fairer horseplay rampage']

In [68]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=10, diversity=0.7)

['increased jax mediocre',
 'wind brothers',
 'seconds brush camouflage',
 'champion ultrashock laser',
 '2116 2337 darius',
 'fairer horseplay rampage',
 'recent nerfs stitching',
 'bonus magic damage',
 'strong tomato solo',
 'jungle karthus dead']